# Fresco Constructor

This code is used for constructing [FRESCO](http://www.fresco.org.uk/) coupled-channel reaction input files.

This code is only suitable for (d,p) reactions.

In [2]:
#imports
import numpy as np
import pandas as pd
import re

### Mass Data Imports

Here we import the [Atomic Mass Evaluation 2020](https://www-nds.iaea.org/amdc/) for use in the reaction codes.

In [4]:
#import NNDC Masses as a Pandas DataFrame
masses = pd.read_fwf("mass_1.mas20.txt",skiprows=33,infer_nrows=3500) 
#infer rows to almost the entire thing so that it parses the larger N,Z,A values correctly


In [5]:
#masses.head()

In [6]:
#drop some columns we don't care about
masses.drop(['1N-Z','O','MASS EXCESS           BINDING ENERGY/A        BETA-DECAY ENERGY'],axis=1,inplace=True)


In [7]:
#drop the units underneath the header keys
masses.drop([0],axis=0,inplace=True)

In [8]:
#masses.head()

In [9]:
#convert the split mass (in micro-amu) into a single micro-amu column
masses['Atomic Mass (micro-amu)'] = masses['ATOMIC MASS'].apply(lambda x: float(x.split("#")[0])) + masses['Unnamed: 7']*1e6
#the split("#") is needed to remove any of the "#" that appear due to estimated errors 

#if *'s are also present in the data:
#masses['Atomic Mass'] = masses['ATOMIC MASS'].apply(lambda x: float(re.split("# *",x)[0])) + masses['Unnamed: 7']*1e6


In [10]:
#create masses in MeV
masses['Atomic Mass (MeV)'] = masses['Atomic Mass (micro-amu)'] *931.5/1.e6 #931.5 MeV = 1 amu. Also convert from micro-amu to amu

In [11]:
masses.head()

,N,Z,A,EL,Unnamed: 7,ATOMIC MASS,Unnamed: 9,Atomic Mass (micro-amu),Atomic Mass (MeV)
1,1.0,0.0,1.0,n,1.0,008664.91590,0.00047,1.008665e+06,939.571369
2,0.0,1.0,1.0,H,1.0,007825.031898,0.000014,1.007825e+06,938.789017
3,1.0,1.0,2.0,H,2.0,014101.777844,0.000015,2.014102e+06,1876.135806
4,2.0,1.0,3.0,H,3.0,016049.28132,0.00008,3.016049e+06,2809.449906
5,1.0,2.0,3.0,He,3.0,016029.32197,0.00006,3.016029e+06,2809.431313


In [131]:
#class for an isotope
#this will hold A,Z,El,mass, etc.
class Isotope:

    def __init__(self,A,El,J,parity,exc_eng=0,q_val = 0,L_trans = 0):
        self.A = A
        self.element = El
        self.Z = get_proton_number(El)
        self.mass = get_isotope_mass(El,A)
        self.mass_amu = self.mass/931.5
        self.spin = J
        self.parity = parity
        self.isotope = str(A)+El
        self.exc_energy = exc_eng
        self.q_value = q_val
        self.L_transfer = L_trans

    def __str__(self):
        return f'''Isotope: {self.isotope}\nA: {self.A}\nElement: {self.element}\nZ: {self.Z}\nMass (MeV): {round(self.mass,3)}
        \rMass (amu): {round(self.mass_amu,3)}\nJ: {self.spin}\nParity: {self.parity}\nExc. energy (MeV): {self.exc_energy}
        \rQ-value (MeV): {self.q_value}\nL-transfer: {self.L_transfer}'''
    

In [133]:
iso = Isotope(87,'Kr',0,-1)

In [135]:
print(iso)

Isotope: 87Kr
A: 87
Element: Kr
Z: 36
Mass (MeV): 80959.79
Mass (amu): 86.913
J: 0
Parity: -1
Exc. energy (MeV): 0
Q-value (MeV): 0
L-transfer: 0


In [15]:
#function for retrieving the isotope mass
def get_isotope_mass(element,A):
    try:
        mass = float(masses[(masses['EL']== element) & (masses['A'] == A)]['Atomic Mass (MeV)'].iloc[0])
        #mass_amu = float(masses[(masses['EL']== element) & (masses['A'] == A)]['Atomic Mass (micro-amu)'].iloc[0])
    except:
        #if the mass is not present in the AME, return 0 to signal to get_user_isotope that the user should try again
        return 0
    else:
        return mass
    
    
    #return float(masses[(masses['EL']== element) & (masses['A'] == A)]['Atomic Mass (MeV)'].iloc[0])
    #.iloc[0] is needed to avoid deprecated TypeError warning


In [16]:
#all of the elements
atomicElements = "H He Li Be B C N O F Ne Na Mg Al Si P S Cl Ar K Ca Sc Ti V Cr Mn Fe Co Ni Cu Zn Ga Ge As Se Br Kr Rb Sr Y Zr Nb Mo Tc Ru Rh Pd Ag Cd In Sn Sb Te I Xe Cs Ba La Ce P Nd Pm Sm Eu Gd Tb Dy Ho Er Tm Yb Lu Hf Ta W Re Os Ir Pt Au Hg Tl Pb Bi Po At Rn Fr Ra Ac Th Pa U Np Pu Am Cm Bk Cf Es Fm Md No Lr Rf Db Sg Bh Hs Mt Ds Rg Cn Nh Fl Mc Lv Ts Og".split(" ")

#function for getting user input for 
def get_user_isotope():
    isotope_mass = 0
    
    while not isotope_mass:
    
        element = "BAD"
        mass_number = -1
        
        #get the element
        while element not in atomicElements:
            element = input("Please enter the name (ex: Kr):" )
        
        #get the mass number
        while mass_number not in range(1,300):
            try:
                mass_number = int(input("Please enter the mass number: "))
            except:
                print("Looks like you did not enter an integer!")
            else:
                if (mass_number not in range(1,300)):
                    print("Mass number out of range!\nMust have mass 1-300")
            
        isotope_mass = get_isotope_mass(element,mass_number)
        #if this returns 0, then the mass is not found in the AME2020.
    

        if not isotope_mass:
            print(f'Isotope {mass_number}{element} does not exist in the AME2020. Try again!')

        #print(f'Selected isotope: {mass_number}{element}, mass {round(isotope_mass,3)} MeV')

    #now return this A,El, mass as a tuple
    return (mass_number,element,isotope_mass)


In [17]:
#kr93 = get_user_isotope()
#kr93

In [18]:
#get the beam energy

def get_user_beam_energy():

    beam_energy = -1
    
    #get the beam energy
    while beam_energy < 0:
        
        try:
            beam_energy = float(input("Enter total beam energy (MeV): "))
        except:
            print("Beam energy is not a number (>0). Try again")
            #continue
        else:
            if (beam_energy > 0):
                return beam_energy
    

In [19]:
#get_user_beam_energy()

In [20]:
#simple function to determine the Z of the nucleus
def get_proton_number(elem):
    return atomicElements.index(elem)+1 #plus one because we index at 0 in the array. 

In [21]:
#get_proton_number('Og')

In [22]:
#initial setup of the FRESCO header info. This is mostly unchanged from reaction to reaction
def construct_fresco_header(lab_energy):

    file = open('build_template_files/NAMELIST.txt','r')
    namelist_header = file.read()
    namelist_header = namelist_header.replace("LAB_ENERGY",str(lab_energy)) #lab energy in MeV
        
    #print(namelist_header)

    return namelist_header


In [23]:
#partition function

def construct_partition(beam,target,reac): #beam is a tuple of A,El,mass
    file = open('build_template_files/PARTITION.txt','r')
    partition_info = file.read()

    #unpack tuples
    #A,El,mass = beam
    #A_targ, El_targ, mass_targ = target
    q_val, n_states = reac

    #reassign parts
    partition_info = partition_info.replace('BEAM_ISOTOPE',beam.isotope)
    #print(type(mass))
    partition_info = partition_info.replace('BEAM_MASS',str(round(float(beam.mass_amu),3)))
    partition_info = partition_info.replace('BEAM_Z', str(beam.Z)) #plus one because we index at 0, but periodic table indexes as t
    partition_info = partition_info.replace('TARGET_NAME',target.isotope)
    partition_info = partition_info.replace('TARGET_MASS',str(round(float(target.mass_amu),3)))
    partition_info = partition_info.replace('TARGET_Z',str(target.Z)) #plus one because of indexing again
    partition_info = partition_info.replace('Q_VALUE',str(q_val))
    partition_info = partition_info.replace('NUM_STATES',str(n_states))
    
    #print(partition_info)
    return partition_info




In [24]:
#construct_partition(('A','Kr','13.23423432'),('B','H','32.323242342'),('1.34','5'))

In [137]:
#state function
def construct_states(beam,target):
    file = open('build_template_files/STATE.txt','r')
    state_info = file.read()


    state_info = state_info.replace('BEAM_J',str(beam.spin))
    state_info = state_info.replace('BEAM_PARITY',str(beam.parity))
    state_info = state_info.replace('TARGET_J',str(target.spin))
    state_info = state_info.replace('TARGET_PARITY',str(target.parity))
    state_info = state_info.replace('Q_VALUE',str(target.q_value))
    
    #print(state_info)
    return state_info


In [153]:
#get the spin-parity for the parent state
def get_user_spin_parity(my_isotope):

    #A,El,mass = beam
   # Z = get_proton_number(El) #find the z of our nucleus

    #if we have an even-even nucleus, return 0+
    #return the form (spin,parity) where parity is defined by +-1
    if (0 == my_isotope.A % 2) and (0 == my_isotope.Z % 2):
        print("Even-even nucleus. Using spin 0+")
        my_isotope.spin = 0
        my_isotope.parity = +1
        return

    #if we have a deuteron, we know what that is too:
    elif (2 == my_isotope.A) and ('H' == my_isotope.element):
        my_isotope.spin = 1
        my_isotope.parity = +1
        return 

    
    else:
        #get a spin that is an whole number of positive half-integer
        get_user_spin(my_isotope)

        
        #now get the parity:
        get_user_parity(my_isotope)

    return


#get spin function
def get_user_spin(isotope):
    #get a spin that is an whole number of positive half-integer
    possible_spins = np.linspace(0,20,41) #unlikely to have a spin higher than 20
    spin = -1
    
    while spin not in possible_spins:
        try:
            spin = float(input("Enter the spin: "))
        except:
            print("Spin was not a number!")
            #continue
        else:
            if (spin not in possible_spins):
                print("Spin is not an non-negative integer or half-integer!")
            else:
                isotope.spin = spin
    return


#get parity function
def get_user_parity(isotope):
    #get the parity:
    parity = 0
    while parity not in (-1,1):
        try:
            parity = int(input(u"Please enter the parity (\u00B11): "))
        except:
            print("Parity is not a number!")
        else:
            if parity not in (-1,1):
                print(u"Parity not in \u00B11")
            else:
                isotope.parity = parity
    return

In [28]:
#function for the number of states
def get_user_number_states():
    num_states = -1
    while num_states < 1:
        try:
            num_states = int(input("Enter the number of states to populate: "))
        except:
            print("Number of states must be a natural number!")
        else:
            if (num_states > 0):
                return num_states

In [29]:
#get_user_number_states()

In [30]:
#function for populating states in the recoiling nucleus
def get_user_excited_state(isotope):

    get_user_excited_state_energy(isotope)
    get_user_spin(isotope)
    get_user_parity(isotope)
    

    return


#get the energy itself
def get_user_excited_state_energy(isotope):

    energy  = -1
    while energy < 0:
        try:
            energy = float(input("Enter the excited-state energy (MeV): "))
        except:
            print("Energy must be a non-negative number!")
        else:
            if (energy >= 0):
                isotope.exc_energy = energy

    return
    


In [79]:
#functions to determine the recoil based on the ejectile
def create_recoil(beam,target,ejectile):

    mass_number = beam.A + target.A - ejectile.A
    proton_number = beam.Z + target.Z -ejectile.Z

    element = atomicElements[proton_number-1] #-1 because we index at 0

    return Isotope(mass_number,element,-1,-1)
    
    

In [175]:
#q-value calculation
def calculate_q_value(beam,target,ejectile,recoil):
    initial_mass = beam.mass + target.mass #in MeV
    final_mass = ejectile.mass + recoil.mass #MeV
    q_val = initial_mass - final_mass #q value is always the same for a given reaction no matter the excitation energy

    recoil.q_value = q_val
    return


In [177]:
#construction of the file in the function here

def Fresco_constructor():

    #First, get the beam that we want for the (d,p) reaction:
    #isotopes are saved as tuples of the form (mass number,element name,mass_MeV)
    user_isotope = get_user_isotope()
    #same, but in amu masses
    #user_isotope_amu = (user_isotope[0],user_isotope[1],user_isotope[2]/931.5)

    
    beam_isotope = Isotope(user_isotope[0],user_isotope[1],-1,-2) #-1,-2 are initializations of the J-pi.

    #get our initial spin-parity values
    get_user_spin_parity(beam_isotope)
    
    #update our spin-parities
    #beam_isotope.spin = beam_jp[0]
    #beam_isotope.parity = beam_jp[1]
    #print(beam_isotope)
    
    user_energy = get_user_beam_energy()

    #we also need to get the deuteron
    target = Isotope(2,'H',1,1) #spin of 1+ for ground-state of deuteron
    #print(deuteron)

    #define our ejectile
    ejectile = Isotope(1,'H',0.5,1) #proton has spin 1/2+

   

    #create the states in the final nucleus here
    num_states = get_user_number_states()
    final_states = []
    for _ in range(num_states):
         #calculate our recoil
        recoil = create_recoil(beam_isotope,target,ejectile)
        get_user_excited_state(recoil)
        calculate_q_value(beam_isotope,target,ejectile,recoil)
        final_states.append(recoil)


    for i in range(len(final_states)):
        print(final_states[i])





    #constructiong of the FRESCO file here
    #create the header (non-fresco) string

    #fresco_string = f' {str(user_isotope[0])+user_isotope[1]}(d,p){str(user_isotope[0]+1)+user_isotope[1]} at {user_energy} MeV'
    fresco_string = f' {beam_isotope.isotope}(d,p){recoil.isotope}'
    #print(fresco_string)

    #add the header that starts the FRESCO initialization
    #fresco_string += '\n' + construct_fresco_header(user_energy)
    #add the first partition for the beam and target
    #fresco_string += '\n\n' + construct_partition(user_isotope_amu,deuteron,(5,2)) #(5,2) just a placeholder for now

    
    #target_jp = get_user_spin_parity(deuteron)

    #fresco_string+='\n\n' + construct_states(beam_jp,target_jp)
    
    
    print('\n')
    print(fresco_string)
    print(beam_isotope)


In [179]:
Fresco_constructor()

Please enter the name (ex: Kr): Kr
Please enter the mass number:  86


Even-even nucleus. Using spin 0+


Enter total beam energy (MeV):  3
Enter the number of states to populate:  1
Enter the excited-state energy (MeV):  0
Enter the spin:  0
Please enter the parity (±1):  1


Isotope: 87Kr
A: 87
Element: Kr
Z: 36
Mass (MeV): 80959.79
Mass (amu): 86.913
J: 0.0
Parity: 1
Exc. energy (MeV): 0.0
Q-value (MeV): 3.290627729627886
L-transfer: 0


 86Kr(d,p)87Kr
Isotope: 86Kr
A: 86
Element: Kr
Z: 36
Mass (MeV): 80025.734
Mass (amu): 85.911
J: 0
Parity: 1
Exc. energy (MeV): 0
Q-value (MeV): 0
L-transfer: 0
